In [ ]:
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.all import *

# Multi-label classification

In [ ]:
path = untar_data(URLs.PASCAL_2007)

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
df['fname']

# Constructing a data block
  - Dataset: A collection that returns a tuple of your independent and dependent variable for a single item
  - DataLoader: An iterator that provides a stream of mini-batches, where each mini-batch is a tuple of a batch of independent variables and a batch of dependent variables
  - fastAI
    - Datasets: An object that contains a training ***Dataset*** and a validation ***Dataset***
    - DataLoaders: An object that contains a training ***DataLoader*** and a validation ***DataLoader***


In [ ]:
a = list(enumerate(string.ascii_lowercase))
a[0], len(a)

In [ ]:
dl_a = DataLoader(a, batch_size= 8, shuffle= True)
b = first(dl_a)
b

In [ ]:
list(zip(b[0], b[1]))

In [ ]:
list(zip(*b))

In [ ]:
a = list(string.ascii_lowercase)
a[0], len(a)

In [ ]:
dss = Datasets(a)
dss[0]

In [ ]:
def f1(o): return o+'a'
def f2(o): return o+'b'


In [ ]:
dss = Datasets(a, [[f1]])
dss[0]

In [ ]:
dss = Datasets(a, [[f1, f2]])
dss[0]

In [ ]:
dss = Datasets(a, [[f1], [f2]])
dss[0]

In [ ]:
dls = DataLoaders.from_dsets(dss, batch_size= 4) 

In [ ]:
first(dls.train)

In [ ]:
dblock = DataBlock()

In [ ]:
dsets = dblock.datasets(df)

In [ ]:
len(dsets.train), len(dsets.valid)

In [ ]:
x, y = dsets.train[0]
x, y

In [ ]:
dblock = DataBlock(get_x= lambda r: r['fname'], get_y = lambda r: r['labels'])
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
def get_x(r): return r['fname']
def get_y(r): return r['labels']
dblock = DataBlock(get_x= get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
Path.BASE_PATH = path

In [ ]:
def get_x(r): return path/'train'/r['fname']
def get_y(r): return r['labels'].split(' ')
dblock = DataBlock(get_x= get_x, get_y= get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
dblock =DataBlock(blocks= (ImageBlock, MultiCategoryBlock), get_x= get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
idxs = torch.where(dsets.train[0][1] == 1.)[0]
dsets.train.vocab[idxs]

In [ ]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y)

dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(df)


In [ ]:
dls.show_batch(nrows= 1, ncols= 3)

In [ ]:
learn = cnn_learner(dls, resnet18)

In [ ]:
x, y = to_cpu(dls.train.one_batch())
activs = learn.model(x)
activs.shape

In [ ]:
activs[0]

In [ ]:
def binary_cross_entropy(inputs, targets):
    inputs = inputs.sigmid()
    return torch.where(targets == 1, 1- inputs, inputs).log().mean()

In [ ]:
loss_func = nn.BCEWithLogitsLoss()
loss = loss_func(activs, y)
loss